In [8]:
import numpy as np
from scipy import sparse
from scipy.sparse import linalg as sp_linalg

In [19]:
def generate_mask(n1, n2, p):
    """
    Generate a mask with probability p for each entry
    :param int n1: number of rows
    :param int n2: number of columns
    :param float p: probability of observing an entry
    """
    omega = np.round(0.5 * (np.random.random((n1, n2)) + p))
    return omega


In [20]:
def generate_ort(p, r):
    E = np.random.randn(p, r)
    E = 1./10 * np.linalg.qr(E)[0]
    return E

In [21]:
# Start parametrs
p = 50
q = 90
r = 5 
prob = 2 * r**2 / (p * q)
K = 10
template = generate_mask(p, q, prob)


In [22]:
#init X, Y, templates
X_0 = np.random.rand(p, q)
eps = np.random.normal(0, 1, (p, q))
omega = generate_mask(p, q, prob)
X = X_0 * omega 
Y = (X_0 + eps) * omega
templates = [generate_mask(p, q, prob) for i in range(K)]

In [13]:
def RGD(Y,r, K, templates, reg=0.05, mu_0=0.5, mu_e=0.5, mu_g=0.5):
    p,q = Y.shape
    
    #init U and V
    z = np.random.rand(K)
    U = generate_ort(p, r)
    V = generate_ort(q, r)
    
    values = np.random.rand(r) 
    L = np.diag(np.sort(values))

    #init E_u and E_v
    E_u = generate_ort(p, r)
    E_v = generate_ort(q, r)

    #init G
    values = np.random.rand(r) 
    sort_values = np.sort(values)[::-1]
    G = np.diag(sort_values)


    iter_num = 0
    max_iter = 30
    I_r = np.eye(r)

    while iter_num < max_iter:
        iter_num += 1
        nabla_z = np.zeros(K)
        nabla_U = 0
        nabla_V = 0
        nabla_l = 0
        for k in range(K):
            image_vector_err = abs(z[k] - np.sum(templates[k] * (U @ L @ V.T)))
            nabla_z[k] += abs(np.sum(Y * templates[k]) - z[k]) - image_vector_err
            nabla_U += image_vector_err * templates[k] @ V @ L
            nabla_V += image_vector_err * templates[k].T @ U @ L
            nabla_l += image_vector_err * U.T @ templates[k] @ V

        nabla_U += - mu_0 * U @ (U.T @ U - I_r) - mu_e * (U - E_u)
        nabla_V += - mu_0 * V @ (V.T @ V - I_r) - mu_e * (V - E_v)
        nabla_l += - mu_g * G @ L

        z = z - reg * nabla_z
        U = U - reg * nabla_U 
        V = V - reg * nabla_V
        L = L - reg * nabla_l


    return U @ L @ V.T
    

In [37]:
X_new = RGD(Y, r, K, templates)

In [38]:
true_error = np.linalg.norm(X_new - X_0, ord='fro') / np.linalg.norm(X_0)
observed_error = np.linalg.norm((X_new - X_0) * omega, ord='fro') / np.linalg.norm(X_0)
print('true error: {}, observed error: {}'.format(true_error, observed_error))

true error: 1.068032045596148, observed error: 0.12116415106913274


In [35]:
X_new_2 = RGD(Y, r, K, templates)

In [36]:
true_error = np.linalg.norm(X_new_2 - X_0, ord='fro') / np.linalg.norm(X_0)
observed_error = np.linalg.norm((X_new_2 - X_0) * omega, ord='fro') / np.linalg.norm(X_0)
print('true error: {}, observed error: {}'.format(true_error, observed_error))

true error: 1.604797374338185e+27, observed error: 2.1756124773619673e+26


Торчовская реализация


In [2]:
import torch

In [3]:
def loss(U, V, Lambda, z, Z, Tau, g, mu_g, mu_o, mu_e, E, E_prime):
    K = Tau.shape[0]
    r = Lambda.shape[0]
    loss1 = -0.5 * torch.norm(Z - z)**2
    loss2 = 0.0
    for k in range(K):
        pred = torch.trace(Tau[k] @ (U @ Lambda @ V.T))
        loss2 -= 0.5 * (z[k] - pred)**2
    loss3 = -0.5 * mu_g * torch.sum(g**2 * torch.diag(Lambda)**2)
    loss4 = -0.5 * mu_o * (torch.norm(U.T @ U - torch.eye(r), 'fro')**2 + torch.norm(V.T @ V - torch.eye(r), 'fro')**2)
    loss5 = -0.5 * mu_e * (torch.norm(U - E, 'fro')**2 + torch.norm(V - E_prime, 'fro')**2)
    
    return loss1 + loss2 + loss3 + loss4 + loss5
  
def gradient_descent(U, V, Lambda_diag, z, Z, Tau, g, mu_g, mu_o, mu_e, E, E_prime, lr=0.01, max_iter=100):
    optimizer = torch.optim.SGD([U, V, Lambda_diag, z], lr=lr, maximize=True)
    
    for i in range(max_iter):
        optimizer.zero_grad()
        Lambda = torch.diag(Lambda_diag)
        current_loss = loss(U, V, Lambda, z, Z, Tau, g, mu_g, mu_o, mu_e, E, E_prime)
        # print(U)
        current_loss.backward()
        optimizer.step()
        print(f"Iteration {i + 1}, Loss: {current_loss.item()}")
    
    return U, V, Lambda_diag, z


p, q, r, K = 10, 10, 3, 4 
#тут нужно U и V сделать ортогональными 
U = torch.randn(p, r, requires_grad=True)
V = torch.randn(q, r, requires_grad=True)
Lambda_diag = torch.linspace(start=1.0, end=0.1, steps=r).requires_grad_()
z = torch.randn(K, requires_grad=True)
Z = torch.randn(K)
Tau = torch.randint(0, 2, (K, p, q), dtype=torch.float32)
g = torch.linspace(start=0.1, end=1.0, steps=r)
mu_g, mu_o, mu_e = 0.1, 0.1, 0.1
E = torch.randn(p, r)
E_prime = torch.randn(q, r)

U_opt, V_opt, Lambda_diag_opt, z_opt = gradient_descent(U, V, Lambda_diag, z, Z, Tau, g, mu_g, mu_o, mu_e, E, E_prime)


Iteration 1, Loss: -69.4010009765625
Iteration 2, Loss: -100.46903991699219
Iteration 3, Loss: -76.88442993164062
Iteration 4, Loss: -24.386783599853516
Iteration 5, Loss: -16.76999282836914
Iteration 6, Loss: -15.50299072265625
Iteration 7, Loss: -14.509227752685547
Iteration 8, Loss: -13.681687355041504
Iteration 9, Loss: -12.972997665405273
Iteration 10, Loss: -12.355308532714844
Iteration 11, Loss: -11.810039520263672
Iteration 12, Loss: -11.323915481567383
Iteration 13, Loss: -10.887015342712402
Iteration 14, Loss: -10.491703033447266
Iteration 15, Loss: -10.131949424743652
Iteration 16, Loss: -9.802913665771484
Iteration 17, Loss: -9.500640869140625
Iteration 18, Loss: -9.221863746643066
Iteration 19, Loss: -8.963849067687988
Iteration 20, Loss: -8.724296569824219
Iteration 21, Loss: -8.501241683959961
Iteration 22, Loss: -8.29300594329834
Iteration 23, Loss: -8.098136901855469
Iteration 24, Loss: -7.915374755859375
Iteration 25, Loss: -7.743617057800293
Iteration 26, Loss: -7.58